In [32]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt

prompt = load_prompt("./prompt.json")

chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

prompt.format(country="Germany")

'What is the capital of Germany?'

In [33]:
prompt = load_prompt("./prompt.yaml")

chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

prompt.format(country="Korea")

'What is the capital of Korea?'

In [40]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.pipeline import PipelinePromptTemplate

chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

intro = PromptTemplate.from_template(
  """
  You are a role playing assistant.
  And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
  """
  This is an example of how you talk:

  Human: {example_question}
  You: {example_answer}
"""
)

start = PromptTemplate.from_template(
  """
  Start now!

  Human: {question}
  You:
"""
)

final = PromptTemplate.from_template(
  """
  {intro}
                                    
  {example}
                            
  {start}
"""
)

prompts = [
  ("intro", intro),
  ("example", example),
  ("start", start),
]

full_prompt = PipelinePromptTemplate(
  final_prompt=final,
  pipeline_prompts=prompts,
)

full_prompt.format(
  character="Pirate",
  example_question="What is your location?",
  example_answer="Arrrg, that is a secret. Arg arg!!",
  question="What is your favorate food?"
)

'\n  \n  You are a role playing assistant.\n  And you are impersonating a Pirate\n\n                                    \n  \n  This is an example of how you talk:\n\n  Human: What is your location?\n  You: Arrrg, that is a secret. Arg arg!!\n\n                            \n  \n  Start now!\n\n  Human: What is your favorate food?\n  You:\n\n'

In [41]:
chain = full_prompt | chat

chain.invoke({
  "character": "Pirate",
  "example_question": "What is your location?",
  "example_answer": "Arrrg, that is a secret. Arg arg!!",
  "question": "What is your favorate food?"
})

Arrr, me favorite food be a good ol' plate o' fish 'n' chips! The taste o' crispy battered fish and salty chips be enough to make me heart sing like a sea shanty! Arg arg!

AIMessageChunk(content="Arrr, me favorite food be a good ol' plate o' fish 'n' chips! The taste o' crispy battered fish and salty chips be enough to make me heart sing like a sea shanty! Arg arg!")